<a href="https://colab.research.google.com/github/jumafernandez/BDM/blob/master/CyA_BD/Conceptos_y_Aplicaciones_Big_Data_UNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TRABAJO INTEGRADOR: CONCEPTOS Y APLICACIONES DE BIG DATA**

__Carrera:__ Espcialización en Inteligencia de Datos orientada a Big Data <br/>
__Docente:__ Waldo Hasperué  <br/>
__Estudiante:__ Juan Manuel Fernandez
<br/>  <br/>
__Resolución de Consultas utilizando PySpark:__ <br />

_Consulta 1:_ Puntaje histórico promedio de cada uno de los arqueros que jugaron en algún equipo de Uruguay. <br />

_Consulta 2:_ ¿Cuál es el nombre del equipo que salio campeón en la temporada 2015 de la primera división de Francia? __(optativo)__ <br />




### Preparación del entorno Pyspark y carga de datos

En primer lugar, cargamos los datos de fantasía desde un repositorio Github:


---



In [6]:
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/liga.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/equipo.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/jugador.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/encuentro.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/participacion_jugador.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/goles.csv

--2019-09-27 12:14:53--  https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/liga.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310 [text/plain]
Saving to: ‘liga.csv’

liga.csv            100%[===================>]     310  --.-KB/s    in 0s      

2019-09-27 12:14:53 (55.3 MB/s) - ‘liga.csv’ saved [310/310]

--2019-09-27 12:14:56--  https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/equipo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358 [text/plain]
Saving to: ‘equipo.csv’

equipo.csv          1

Instalamos la librería pyspark:

In [7]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 118kB/s 
     |████████████████████████████████| 204kB 45.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=1aaafcec5294be00f41a333cb5bbed8bd8c0d0033a4f20791c56e3edc66bbdd9
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


Ahora, generamos el entorno PySpark cargando las librerías a utilizar, instanciamos el Spark Context y se da formato columnar a los datos:

In [8]:
# Importo las librerias
from pyspark import SparkContext, SQLContext

# Genero el Spark Context y el SQLContext para la DB 
sc = SparkContext("local", "Simple App") 

# Cargo SQLContext, solo para instanciar dataframes, no para consultas
df = SQLContext(sc)

# Defino el path donde estan los archivos (Solo para máquina local) 
data_path = ''

# Cargo los RDD con los datos de los archivos y acomodo los datos
RDD_liga = sc.textFile(data_path + "liga.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), int(d[1]), d[2].replace('"', ''), d[3].replace('"', '')))
RDD_liga.first()

RDD_equipo = sc.textFile(data_path + "equipo.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), d[1].replace('"', ''), d[2].replace('"', ''), d[3].replace('"', ''), d[4].replace('"', '')))
RDD_equipo.first()

RDD_jugador = sc.textFile(data_path + "jugador.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), d[1].replace('"', ''), int(d[2]), d[3].replace('"', '')))
RDD_jugador.first()

RDD_encuentro = sc.textFile(data_path + "encuentro.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), int(d[1]), d[2].replace('"', ''), int(d[3]), int(d[4]), d[5].replace('"', ''), float(d[6]), int(d[7])))
RDD_encuentro.first()

RDD_participacion_jugador = sc.textFile(data_path + "participacion_jugador.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id_jugador"').\
           map(lambda d: (int(d[0]), int(d[1]), int(d[2]), int(d[3]), int(d[4]), int(d[5]), int(d[6])))
RDD_participacion_jugador.first()

RDD_goles = sc.textFile(data_path + "goles.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), int(d[1]), int(d[2]),d[3].replace('"', ''), int(d[4])))
RDD_goles.first()


(1, 2, 2, 'CABEZA', 90)

Luego, los transformo a dataframes por dos cuestiones centrales:
1. _Conflictos con el Join en los RDD:_ Cuando intento aplicar un join sobre un RDD puro, me proyecta solo un subconjuto de los atributos.
2. _Comodidad:_ trabajando con dataframes, puedo referenciar los datos por nombre de columnas, lo cual brinda mayor expresividad y facilidad para la resolución de las consultas.

Independientemente de esto, no se utilizarán las otras características de SparkSQL (consultas SQL) en la resolución del trabajo.

In [0]:
# Creo los dataframes
df_ligas = df.createDataFrame(RDD_liga,['id','temporada', 'division', 'pais'])
df_equipos =  df.createDataFrame(RDD_equipo,['id','nombre', 'pais', 'ciudad', 'fundacion'])
df_jugadores = df.createDataFrame(RDD_jugador,['id','nombre', 'edad', 'posicion'])
df_encuentros = df.createDataFrame(RDD_encuentro,['id','id_liga', 'fecha', 'id_equipo_local', 'id_equipo_visitante', 'estadio', 'recaudacion', 'localidades_vendidas'])
df_participaciones = df.createDataFrame(RDD_participacion_jugador,['id_jugador','id_equipo', 'id_partido', 'puntaje_recibido', 'cantidad_amarillas', 'cantidad_rojas', 'minutos_jugados'])
df_goles = df.createDataFrame(RDD_goles,['id', 'id_jugador', 'id_partido', 'tipo_gol', 'minuto_gol'])


### Resolución Consulta 1: Puntaje histórico promedio de cada uno de los arqueros que jugaron en algún equipo de Uruguay.

Para comenzar con la resolución, definimos parámetros para flexibilizar la consulta (posición y país):


In [0]:
posicion = 'ARQUERO'
pais_filtro = 'ARGENTINA'

Tomo los equipos de la variable _pais_filtro_ y los jugadores que juegan en el conteenido de la variable _posicion_:


In [0]:
equipos_pais = df_equipos.filter(df_equipos.pais==pais_filtro)
jugadores_posicion = df_jugadores.filter(df_jugadores.posicion==posicion)

A continuación, resuelvo el resto de la consulta tomando las participaciones de los arqueros en equipos uruguayos y agrupando las instancias aplicando la función de agregación MEAN:

In [11]:
# Joineo ambas cosas
participaciones = jugadores_posicion.join(df_participaciones, jugadores_posicion.id==df_participaciones.id_jugador)

# Tomo las columnas que me interesan y muestro el resultado agrupando
jugadores_puntaje = participaciones.select('nombre', 'puntaje_recibido')
jugadores_puntaje.groupby('nombre').agg({'puntaje_recibido': 'mean'}).show()

+----------------+---------------------+
|          nombre|avg(puntaje_recibido)|
+----------------+---------------------+
|ANDRADA, ESTEBAN|                  6.0|
+----------------+---------------------+

